## A simple model for demand and supply of publicly-provided services in a city

### Demand modelling

In [1]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [2]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [3]:
from src.models.cityItems import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [4]:
AgeGroup.classify_array(range(18))

[<AgeGroup.Newborn: (0, 3)>,
 <AgeGroup.Newborn: (0, 3)>,
 <AgeGroup.Newborn: (0, 3)>,
 <AgeGroup.Kinder: (3, 6)>,
 <AgeGroup.Kinder: (3, 6)>,
 <AgeGroup.Kinder: (3, 6)>,
 <AgeGroup.ChildPrimary: (6, 10)>,
 <AgeGroup.ChildPrimary: (6, 10)>,
 <AgeGroup.ChildPrimary: (6, 10)>,
 <AgeGroup.ChildPrimary: (6, 10)>,
 <AgeGroup.ChildMid: (10, 15)>,
 <AgeGroup.ChildMid: (10, 15)>,
 <AgeGroup.ChildMid: (10, 15)>,
 <AgeGroup.ChildMid: (10, 15)>,
 <AgeGroup.ChildMid: (10, 15)>,
 <AgeGroup.ChildHigh: (15, 19)>,
 <AgeGroup.ChildHigh: (15, 19)>,
 <AgeGroup.ChildHigh: (15, 19)>]

In [ ]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: list(map(geopy.Point, list(zip(np.round(np.random.uniform(45.40, 45.50, n), 5), 
                                np.round(np.random.uniform(9.1, 9.3, n), 5)))))
make_shapely_point = lambda geoPoint: shapely.geometry.Point((geoPoint.longitude, geoPoint.latitude))

In [ ]:
### Demand modelling
class DemandUnit:
    def __init__(self, position=None, agesInput=None, attributesIn={}):
        # make defaults
        if not position: position = get_random_pos(1)[0]
        if not agesInput: agesInput = {a: 1 for a in AgeGroup.all()}
        assert isinstance(position, geopy.Point), 'Position must be a geopy Point'
        assert isinstance(attributesIn, dict), 'Attributes can be provided in a dict'
        # expand input to all age group keys and assign default 0 to missing ones
        self.ages = {a: agesInput.get(a, 0) for a in AgeGroup.all()}
        self.position = position
        self.polygon = attributesIn.get('geometry', [])
        # precompute export format for speed
        self.export = pd.DataFrame(self.ages, index=(tuple(self.position)))

In [ ]:
def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    """ """
    # initialise output
    outDemand = dict()
    # consolidate positions. If two households share the same position, sum components.
    householdData = pd.concat([h.export for h in householdList])
    householdData['position'] = householdData.index 
    consolidated = householdData.groupby('position').sum()
    
    for thisServType in outputServices:
        outDemand[thisServType] = consolidated*thisServType.demandFactors
        
    return outDemand

In [ ]:
hhList =  [DemandUnit() for i in range(40)]
#evaluate_demand(hhList)

In [ ]:
## Matching demand and supply
def get_satisfaction_indexes(householdList):
    pass